In [95]:
import pybullet
import gym
import numpy as np
import matplotlib.pyplot as plt
import pybullet_data
import math
import rrtstar
import mpc
import time
import torch

In [ ]:
#!git clone https://github.com/ROBOTIS-GIT/turtlebot3.git

In [67]:
# Initializing pybullet
physicsClient = pybullet.connect(pybullet.GUI)
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
pybullet.setGravity(0,0,-9.8)

# Loading chessgrid plane
planeID = pybullet.loadURDF('plane.urdf')
chess_grid = pybullet.loadTexture('checker_huge.gif')
pybullet.changeVisualShape(planeID, -1, textureUniqueId=chess_grid)




# Creating robots
player = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.25,0.25,0], pybullet.getQuaternionFromEuler([0,0,0]))

start = (0.25,0.25)

static = []

#static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,0.25,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
#static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,0.75,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
#static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,-0.25,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
#static.append(pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [1.25,0.75,0], pybullet.getQuaternionFromEuler([0,0,math.pi])))
#target = static[3]

goal = (1.25, 0.75)

# For milestone 2 only - setting all non-player targets to have no collision with player robot
# Doesn't work when dragging into each other but might still work?
#pybullet.setCollisionFilterGroupMask(player, target, 0, 0)

#for robot in static:
#    pybullet.setCollisionFilterGroupMask(player, robot, 0, 0)

# Setting colors to distinguish robots. BLUE for player, GREEN for obstacle, RED for target
pybullet.changeVisualShape(player, 0, rgbaColor=[0,0,1,1])
#for robot in static:
#    pybullet.changeVisualShape(robot, 0, rgbaColor=[0,1,0,1])
#pybullet.changeVisualShape(target, 0, rgbaColor=[1,0,0,1])



# Setting up overhead camera
viewMatrix = pybullet.computeViewMatrix(
            cameraEyePosition=[0.75,0.25,3],
            cameraTargetPosition=[0.75,0.25,0],
            cameraUpVector=[0,1,0])

projectionMatrix = pybullet.computeProjectionMatrixFOV(
            fov=45.0,
            aspect=1.0,
            nearVal=0.1,
            farVal=3.1)

width, height, rgbImg, depthImg, segImg = pybullet.getCameraImage(width = 224, height = 224, viewMatrix=viewMatrix, projectionMatrix = projectionMatrix)

# Getting position and orientation
print(pybullet.getBasePositionAndOrientation(player))
#print(pybullet.getBasePositionAndOrientation(static[0]))
#print(pybullet.getBasePositionAndOrientation(static[1]))
#print(pybullet.getBasePositionAndOrientation(static[2]))
#print(pybullet.getBasePositionAndOrientation(target))


# Start simulation
pybullet.setRealTimeSimulation(1)

((0.25, 0.25, 0.0), (0.0, 0.0, 0.0, 1.0))


In [68]:
# RRT
samplingArea = [0.25, 1.25, -0.25, 0.75]
RRT = rrtstar.RRT(start, goal, [], samplingArea, 'rrtstar', 'rectangle', 2)
path = RRT.planning()

[0.25, 0.25, 0]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.3187545914968042, 0.003687417110986857, 1.666070827354531]
[0. 0. 0.]
[0.3187545914968042, 0.003687417110986857, 1.666070827354531]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.3187545914968042, 0.003687417110986857, 1.666070827354531]
[0. 0. 0.]
[0.6134252622959829, -0.05556737798419731, 5.476282084123041]
[0. 0. 0.]
[0.6134252622959829, -0.05556737798419731, 5.476282084123041]
[0. 0. 0.]
[0.3187545914968042, 0.003687417110986857, 1.666070827354531]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.3187545914968042, 0.003687417110986857, 1.666070827354531]
[0. 0. 0.]
[0.6134252622959829, -0.05556737798419731, 5.476282084123041]
[0. 0. 0.]
[0.7051124836976996, -0.1773114635079046, 1.1630327656067379]
[0. 0. 0.]
[0.7051124836976996, -0.1773114635079046, 1.1630327656067379]
[0. 0. 0.]
[0.7051124836976996, -0.1773114635079046, 1.1630327656067379]
[0. 0. 0.]
[0.3187545914968042, 0.003687417110986857, 1.666070827354531]
[0. 0. 0.]
[0.25, 0.25, 0]


[0.8673252914195165, 0.2868906293275598, 1.8826553627356166]
[0. 0. 0.]
[0.9981068176229224, 0.19394240854710654, 5.9410079106255775]
[0. 0. 0.]
[0.8591104414992935, 0.7401579712799627, 0.35992452044333934]
[0. 0. 0.]
[0.2546991776262081, 0.30084816084506616, 0.3269499490723896]
[0. 0. 0.]
[0.2546991776262081, 0.30084816084506616, 0.3269499490723896]
[0. 0. 0.]
[0.2546991776262081, 0.30084816084506616, 0.3269499490723896]
[0. 0. 0.]
[0.2546991776262081, 0.30084816084506616, 0.3269499490723896]
[0. 0. 0.]
[0.2546991776262081, 0.30084816084506616, 0.3269499490723896]
[0. 0. 0.]
[0.2546991776262081, 0.30084816084506616, 0.3269499490723896]
[0. 0. 0.]
[0.2546991776262081, 0.30084816084506616, 0.3269499490723896]
[0. 0. 0.]
[0.2546991776262081, 0.30084816084506616, 0.3269499490723896]
[0. 0. 0.]
[0.2546991776262081, 0.30084816084506616, 0.3269499490723896]
[0. 0. 0.]
[0.2546991776262081, 0.30084816084506616, 0.3269499490723896]
[0. 0. 0.]
[0.2546991776262081, 0.30084816084506616, 0.32694994

[0.5013291590797426, -0.1844451573836936, 6.132063666009565]
[0. 0. 0.]
[0.5013291590797426, -0.1844451573836936, 6.132063666009565]
[0. 0. 0.]
[0.5013291590797426, -0.1844451573836936, 6.132063666009565]
[0. 0. 0.]
[0.5013291590797426, -0.1844451573836936, 6.132063666009565]
[0. 0. 0.]
[0.5013291590797426, -0.1844451573836936, 6.132063666009565]
[0. 0. 0.]
[0.8673252914195165, 0.2868906293275598, 1.8826553627356166]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.3187545914968042, 0.003687417110986857, 1.666070827354531]
[0. 0. 0.]
[0.6134252622959829, -0.05556737798419731, 5.476282084123041]
[0. 0. 0.]
[0.7051124836976996, -0.1773114635079046, 1.1630327656067379]
[0. 0. 0.]
[0.556263163576679, 0.745950135809089, 1.595936645428458]
[0. 0. 0.]
[1.0281476939752654, 0.250692086734561, 4.466444341700954]
[0. 0. 0.]
[1.25, 0.75, 0]
[0. 0. 0.]
[0.3628243090682798, 0.29665229370386137, 2.4294283968716153]
[0. 0. 0.]
[0.8673252914195165, 0.2868906293275598, 1.8826553627356166]
[0. 0. 0.]
[0.99810681

[0.556263163576679, 0.745950135809089, 1.595936645428458]
[0. 0. 0.]
[1.0281476939752654, 0.250692086734561, 4.466444341700954]
[0. 0. 0.]
[1.25, 0.75, 0]
[0. 0. 0.]
[0.3628243090682798, 0.29665229370386137, 2.4294283968716153]
[0. 0. 0.]
[0.8673252914195165, 0.2868906293275598, 1.8826553627356166]
[0. 0. 0.]
[0.9981068176229224, 0.19394240854710654, 5.9410079106255775]
[0. 0. 0.]
[0.8591104414992935, 0.7401579712799627, 0.35992452044333934]
[0. 0. 0.]
[0.2546991776262081, 0.30084816084506616, 0.3269499490723896]
[0. 0. 0.]
[0.2962392411577539, 0.7117212406843686, 3.6720287627197843]
[0. 0. 0.]
[0.940181742387967, 0.5546529560375042, 4.499447475576231]
[0. 0. 0.]
[0.6903138343326441, -0.06720881562023584, 2.4576152911191382]
[0. 0. 0.]
[0.5013291590797426, -0.1844451573836936, 6.132063666009565]
[0. 0. 0.]
[0.9507699182356522, -0.11962018027023968, 1.8732879531229105]
[0. 0. 0.]
[0.8781351711524435, 0.09436978641445604, 5.388366998168661]
[0. 0. 0.]
[0.6534262156799655, 0.2271835084270

[0. 0. 0.]
[0.5771941985210881, -0.18097466045182975, 5.709940176921651]
[0. 0. 0.]
[0.5771941985210881, -0.18097466045182975, 5.709940176921651]
[0. 0. 0.]
[0.5771941985210881, -0.18097466045182975, 5.709940176921651]
[0. 0. 0.]
[0.5771941985210881, -0.18097466045182975, 5.709940176921651]
[0. 0. 0.]
[0.5771941985210881, -0.18097466045182975, 5.709940176921651]
[0. 0. 0.]
[0.5771941985210881, -0.18097466045182975, 5.709940176921651]
[0. 0. 0.]
[0.5771941985210881, -0.18097466045182975, 5.709940176921651]
[0. 0. 0.]
[0.5771941985210881, -0.18097466045182975, 5.709940176921651]
[0. 0. 0.]
[0.7602520300625009, 0.7435604309523215, 2.5560738297316683]
[0. 0. 0.]
[0.25, 0.25, 0]
[0. 0. 0.]
[0.3187545914968042, 0.003687417110986857, 1.666070827354531]
[0. 0. 0.]
[0.6134252622959829, -0.05556737798419731, 5.476282084123041]
[0. 0. 0.]
[0.7051124836976996, -0.1773114635079046, 1.1630327656067379]
[0. 0. 0.]
[0.556263163576679, 0.745950135809089, 1.595936645428458]
[0. 0. 0.]
[1.028147693975265

...

[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614

[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[0.4792514842164064, -0.171

[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.005

[1.112789105978523, 0.2363214409719787, 3.205792765103424]
[0. 0. 0.]
[1.1980242076503071, 0.0722658136460187, 0.34314620335694745]
[0. 0. 0.]
[0.8621024215440835, 0.16578963653198397, 1.4148096242075732]
[0. 0. 0.]
[0.5272076713426961, 0.24491669518657366, 0.7820224288546976]
[0. 0. 0.]
[0.8508734356690694, 0.29771146517613767, 1.3093614004533065]
[0. 0. 0.]
[0.4792514842164064, -0.17135210851323812, 1.6449870341996249]
[0. 0. 0.]
[1.005406536013541, 0.4329953490124534, 3.1916422606340196]
[0. 0. 0.]
[1.1831944406217847, -0.17683932215314846, 5.477954447236405]
[0. 0. 0.]
[1.1831944406217847, -0.17683932215314846, 5.477954447236405]
[0. 0. 0.]
[1.1831944406217847, -0.17683932215314846, 5.477954447236405]
[0. 0. 0.]
[1.1831944406217847, -0.17683932215314846, 5.477954447236405]
[0. 0. 0.]
[1.1831944406217847, -0.17683932215314846, 5.477954447236405]
[0. 0. 0.]
[1.1831944406217847, -0.17683932215314846, 5.477954447236405]
[0. 0. 0.]
[1.1831944406217847, -0.17683932215314846, 5.4779544472

[0.5444076596288461, -0.16901206549318737, 1.7679176197718083]
[0. 0. 0.]
[0.9801337312847905, -0.057115539410446, 1.0301971595266084]
[0. 0. 0.]
[1.1739893908422039, 0.09890793524983044, 5.970233284109406]
[0. 0. 0.]
[0.8613733978091892, 0.24098016007213208, 3.117883116562254]
[0. 0. 0.]
[0.9323462764401845, -0.1451413864542388, 0.8292755403459531]
[0. 0. 0.]
[0.7778706907630664, 0.2874633133598611, 3.8672422395961474]
[0. 0. 0.]
[0.6121265969368433, 0.6679403356258627, 2.7224452766783602]
[0. 0. 0.]
[0.7785513711954236, 0.650981787152663, 5.72050514363101]
[0. 0. 0.]
[1.0925306186192678, -0.03932863404344489, 5.6629626784776725]
[0. 0. 0.]
[1.1144343031658517, 0.7367639189580714, 1.4265835999478185]
[0. 0. 0.]
[0.935528770274545, 0.40927726946182286, 3.2326036269361182]
[0. 0. 0.]
[0.8165174381658721, 0.06660941380093133, 0.5965363333598275]
[0. 0. 0.]
[0.831767396181575, 0.73535020450551, 0.5736165392105608]
[0. 0. 0.]
[1.2492976461498344, 0.4887611534895132, 3.930999168988139]
[0. 

In [69]:
print(path)
path = np.flip(path, 0)
print(path)


[[1.25, 0.75, 0], [1.25, 0.75, 0], [0.25, 0.25, 0]]
[[0.25 0.25 0.  ]
 [1.25 0.75 0.  ]
 [1.25 0.75 0.  ]]


In [ ]:
extended_path = 

In [210]:
def applyAction(actions, orig_state):
    new_states = []
    for i in range(G):
        action_set = actions[i]
        for j in range(H):
            action = action_set[j]
            action_3d = [fM * action[0], fM * action[1], 0]
            for x in range(25):
                pybullet.stepSimulation()
                time.sleep(1./240.)
                current_pos = pybullet.getBasePositionAndOrientation(player)[0]
                pybullet.applyExternalForce(player, -1, action_3d, current_pos, pybullet.WORLD_FRAME)
            #pybullet.resetBaseVelocity(player, action_3d)
        
        new_states.append(pybullet.getBasePositionAndOrientation(player))
        pybullet.resetBasePositionAndOrientation(player, orig_state[0], orig_state[1])
    return new_states
        

In [211]:
def getCost(states, target):
    target_pos = [target[0], target[1]]
    target_angle = target[2]
    costs = []
    for i in range(G):
        current = states[i]
        current_pos = current[0]
        current_angle = pybullet.getEulerFromQuaternion(current[1])[2]
        # Double check above. Based on angle used for initial setting position
        cost = math.sqrt(((current_pos[0] - target_pos[0]) ** 2) + ((current_pos[1] - target_pos[1]) ** 2))
        cost += abs(current_angle - target_angle)
        costs.append(cost)
    return costs

In [212]:
H = 5
G = 10
T = 5
K = 1
fM = 50

def cem(path):
    pybullet.setRealTimeSimulation(0)
    for i in range(len(path) - 1):
        initial_actions = np.random.normal(0,1,(G,H,2))
        state = pybullet.getBasePositionAndOrientation(player)
        print(state)
        for t in range(T):
            new_states = applyAction(initial_actions, state)
            costs = getCost(new_states, path[i + 1])
            sorted_actions = [act for _,act in sorted(zip(costs, initial_actions), key=lambda pair: pair[0])]
            new_mean = np.mean(sorted_actions[:K])
            new_std = np.std(sorted_actions[:K])
            new_actions = np.random.normal(new_mean, new_std, (G,H,2))
            #print(sorted_actions[:K])
            #print(new_actions)
            initial_actions = np.concatenate((sorted_actions[:K], new_actions))
            
        best_action = initial_actions[0][0] # Because no horizon length
        action = [fM * best_action[0], fM * best_action[1], 0]
        print(action)
        for x in range(25):
            pybullet.stepSimulation()
            time.sleep(1./240.)
            pybullet.applyExternalForce(player, -1, action, state[0], pybullet.WORLD_FRAME)
        #pybullet.resetBaseVelocity(player, action)

            
            


In [214]:
cem(path)
pybullet.setRealTimeSimulation(1)

((0.24524251188487425, 0.24995636494076864, 0.0014314068876408873), (-1.897432712082751e-05, 0.08220453672759859, 0.000279951157589819, 0.9966154400814414))
[54.94037549613054, -38.35496888519001, 0]
((0.2916564882272533, 0.2436232347869599, 0.0010209576937640454), (-0.0015584078386404686, 0.034263766293042175, 0.04301457777153208, 0.9984855090503623))
[45.017443797338345, 59.61828061618524, 0]


In [213]:
pybullet.resetBasePositionAndOrientation(player, [0.25,0.25,0], pybullet.getQuaternionFromEuler([0,0,0]))

In [215]:
pybullet.disconnect()